In [ ]:
#Login to controller
import prisma_sase
import time
import pandas as pd
import termtables as tt
from time import sleep

PRISMASASE_CLIENT_ID = "xxxxxx"
PRISMASASE_CLIENT_SECRET = "xxxxxx"
 
tsgid = "xxxxxx"
sdk = prisma_sase.API(controller="https://api.sase.paloaltonetworks.com", ssl_verify=False)
sdk.set_debug(3)
sdk.interactive.login_secret(client_id=PRISMASASE_CLIENT_ID, client_secret=PRISMASASE_CLIENT_SECRET, tsg_id=tsgid) 
sdk.remove_header('X-PANW-Region')

In [ ]:
#fetch all the compute locations
from geopy import distance

#Fetch all the locations
url = "https://api.sase.paloaltonetworks.com/sse/config/v1/locations"
sdk.remove_header('X-PANW-Region')
resp = sdk.rest_call(method="GET",url=url)
prisma_sase.jd_detailed(resp)

print(resp.json())
locations = resp.json()

In [ ]:
#Fetch all the RN Sites
sites = [
    {
        "name": "London",
        "latitude": 51.507351,
        "longitude": -0.127758,
        "bandwidth": 50,
        "subnets": ["10.2.2.0/24"],
        "redundancy": False,
        "bgp": True,
        "platform": "paloalto"
    },
    {
        "name": "Tokyo",
        "latitude": 35.689487,
        "longitude": 139.691711,
        "bandwidth": 50,
        "subnets": ["10.6.2.0/24"],
        "redundancy": True,
        "bgp": False,
        "platform": "velocloud"
    }
]

#Compare the distances between RN Sites and the locations and fetch the nearest region

for site in sites:
    distances = {}
    for l in locations:
        d = distance.distance((site["latitude"],site["longitude"]), (l["latitude"],l["longitude"])).km
        distances[l["value"]] = d
    nearest_region = min(distances, key= distances.get)


    #Check the BW available in the nearest region
    for region in locations:
        if region["region"] == nearest_region:
            nearest_aggregate = region.get("aggregate_region")
            break

    #Fetch the allocated BW in the nearest aggregate
    url = "https://api.sase.paloaltonetworks.com/sse/config/v1/bandwidth-allocations"
    resp = sdk.rest_call(method="GET",url=url)
    prisma_sase.jd_detailed(resp)

    nearest_regions_present = resp.json()["data"]
    print(nearest_aggregate)
    print("\n")
    print(nearest_regions_present)
    for nearest_region_present in nearest_regions_present:
        if nearest_region_present["name"] == nearest_aggregate:
            #Check the BW allocated
            allocated_bw = nearest_region_present["allocated_bandwidth"]
            #fetch the SPN List of this nearest region
            print(nearest_region_present["spn_name_list"])
            spn_list = nearest_region_present["spn_name_list"]

            #Print the list of Site to Region mapping
            print("\n SITE                     {} ".format(site))
            print("\n REGION                   {} ".format(nearest_aggregate))
            print("\n CURRENT ALLOCATED BW     {} ".format(allocated_bw))
            print("\n SPN List                 {} ".format(spn_list))

In [ ]:
#Agg BW Allocation via API
url = "https://api.sase.paloaltonetworks.com/sse/config/v1/bandwidth-allocations"
payload = {
    "name":agg_bw_reg_name,
    "allocated_bandwidth":agg_bw,
        
}
sdk_resp = sdk.rest_call(url=url, data=payload, method="POST")  
resp = sdk_resp.json()
try:
    reg_name = resp["name"]
except:
    print("BW Allocation not successful")
    exit(0)
#print(resp)

print("BW Allocation Successful")
Header = ["Agg BW Region", "BW Allocated", "SPN Node"]
RList = []
   
RList.append([resp["name"],resp["allocated_bandwidth"],resp["spn_name_list"][0]])
    
table_string = tt.to_string(RList, Header, style=tt.styles.ascii_thin_double)
print(table_string)

In [ ]:

#Create IKE GW
ike_payload_list = [
    {
        "name":"IKE-GW",
        "folder":"Remote Networks",
        "local_address":{"interface":"vlan"},
        "authentication":{"pre_shared_key":{"key":"paloalto"}},
        "peer_address":{"dynamic":{}},
        "local_id":{"type":"ipaddr","id":"1.1.1.1"},
        "peer_id":{"type":"ipaddr","id":"2.2.2.2"},
        "protocol":{
            "version":"ikev2-preferred",
            "ikev1":
            {"ike_crypto_profile":"PaloAlto-Networks-IKE-Crypto",
             "dpd":{"enable":true}},
            "ikev2":
            {"dpd":{"enable":true}}
        },
        "protocol_common":{"nat_traversal":{"enable":true},
        "fragmentation":{"enable":false}}
    },
        {  
        "name":"IKE-GW2",
        "folder":"Remote Networks",
        "local_address":{"interface":"vlan"},
        "peer_address":{"dynamic":{}},
        "authentication":{"pre_shared_key":{"key":"cloudgen!x123"}},
        "peer_id":{"type":"ufqdn","id":"cgx-tunnel@mycompany.com"},
        "local_id":{"type":"ufqdn","id":"prisma-tunnel@mycompany.com"},
        "protocol":{
            "version":"ikev2",
            "ikev1":{"dpd":{"enable":true}},
            "ikev2":{"dpd":{"enable":true},"ike_crypto_profile":"CloudGenix-IKE-Crypto-Default"}
        },
        "protocol_common":{"nat_traversal":{"enable":true},
        "fragmentation":{"enable":false},
        "passive_mode":true
        }
    }
]
ike_gw_url = "https://api.sase.paloaltonetworks.com/sse/config/v1/ike-gateways?folder=Remote Networks"

for ike_payload in ike_payload_list:
    resp = sdk.rest_call(url=ike_gw_url, data=ike_payload, method="POST")
    prisma_sase.jd_detailed(resp)


In [ ]:
#Create IPSec Tunnel
ipsec_payload_list = [
    {
        "name":"RN-Tunnel",
        "folder":"Remote Networks",
        "tunnel_interface":"tunnel",
        "auto_key":
            {
                "ike_gateway":[{"name":"IKE-GW"}],
                "ipsec_crypto_profile":"PaloAlto-Networks-IPSec-Crypto"
            },
        "anti_replay":true
    },
    {
        "name":"RN-Tunnel2",
        "folder":"Remote Networks",
        "tunnel_interface":"tunnel",
        "auto_key":
            {
                "ike_gateway":[{"name":"IKE-GW2"}],
                "ipsec_crypto_profile":"CloudGenix-IPSec-Crypto-Default"
            },
            "anti_replay":true,
            "copy_tos":true,
            "tunnel_monitor":   
                {
                    "enable":true,
                    "destination_ip":"172.16.0.2"
                }
    }
]
ipsec_tunnel_url="https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-tunnels?folder=Remote Networks"

for ipsec_payload in ipsec_payload_list:
    resp = sdk.rest_call(url=ipsec_tunnel_url, data=ipsec_payload, method="POST")
    prisma_sase.jd_detailed(resp)

In [ ]:
#Create RN Network
RN_payload_list = [
    {
        "name":"RN-Tunnel",
        "folder":"Remote Networks",
        "tunnel_interface":"tunnel",
        "auto_key":
            {
                "ike_gateway":[{"name":"IKE-GW"}],
                "ipsec_crypto_profile":"PaloAlto-Networks-IPSec-Crypto"
            },
        "anti_replay":true
    },
    {
        "name":"RN-Tunnel2",
        "folder":"Remote Networks",
        "tunnel_interface":"tunnel",
        "auto_key":
            {
                "ike_gateway":[{"name":"IKE-GW2"}],
                "ipsec_crypto_profile":"CloudGenix-IPSec-Crypto-Default"
            },
            "anti_replay":true,
            "copy_tos":true,
            "tunnel_monitor":   
                {
                    "enable":true,
                    "destination_ip":"172.16.0.2"
                }
    }
]
RN_url="https://api.sase.paloaltonetworks.com/sse/config/v1/ipsec-tunnels?folder=Remote Networks"

for RN_payload in RN_payload_list:
    resp = sdk.rest_call(url=RN_url, data=RN_payload, method="POST")
    prisma_sase.jd_detailed(resp)